In [ ]:
# main.py
from initializations import Initializations
from trading_data import TradingData
from order_executor import OrderExecutor, get_token
from post_order_processor import PostOrderProcessor
import datetime
import time
import logging

# グローバルな設定
API_PASSWORD = "1995taka"
ORDER_PASSWORD = "1995tAkA@@"

#インデックス 295~300
SYMBOL = "1579"

EXCHANGE = 1  # 東証プライム市場を設定
SLEEP_INTERVAL = 0.3 # 待機時間（秒）

def setup_logger():
    """
    ロガーを設定します。
    """
    logger = logging.getLogger('TradingLogger')
    logger.setLevel(logging.INFO)
    
    # 既にハンドラーが設定されている場合は追加しない
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    # ロガーの伝播を無効化
    logger.propagate = False
    return logger

def main():
    # 初期化クラスのインスタンスを作成
    init = Initializations()
    init.order_password = ORDER_PASSWORD
    init.symbol = SYMBOL
    init.exchange = EXCHANGE
    # ロガーの設定
    init.logger = setup_logger()
    # トークンを取得
    token = get_token(API_PASSWORD)
    init.token = token
    # TradingDataクラスのインスタンスを作成
    trading_data = TradingData(init, token)
    # OrderExecutorクラスのインスタンスを作成
    order_executor = OrderExecutor(init, trading_data, token, ORDER_PASSWORD)
    # PostOrderProcessorクラスのインスタンスを作成
    post_order_processor = PostOrderProcessor(init)

    # 初回の価格取得
    initial_price = trading_data.fetch_current_price()
    
    init.previous_price = initial_price
    init.current_price = initial_price
    init.logger.info(f"初回価格を設定しました。価格: {initial_price}")
    
    # トレーディングループを開始
    while True:
        # 最新の株価データを取得
        current_price = trading_data.fetch_current_price()
        # price = (current_price)
        if current_price is not None:
            # データが4つになったらOHLCを生成
            if len(init.prices) >= 4:
                trading_data.create_ohlc()
                # Buy and Holdのエクイティカーブを計算
                trading_data.calculate_buy_and_hold_equity()
                # テクニカル指標の計算
                trading_data.calculate_technical_indicators()
                # 最新のテクニカル指標データを取得
                band_width = init.df['band_width']
                hist = init.df['hist']
                di_difference = init.df['di_difference']
                adx_difference = init.df['adx_difference']
                trading_data.update_latest_9_data(band_width, hist, di_difference, adx_difference)
                init.logger.info(f"Interpolated data length: {len(init.interpolated_data)}")
                # トレーディング状態の計算と更新
                post_order_processor.calculate_trading_values(datetime.datetime.now())
                # シグナルを生成
                trading_data.generate_signals(init.interpolated_data, init.R1, init.R2, init.R3, init.S1, init.S2, init.S3)
                # 注文の実行
                order_executor.execute_orders()
                # 補間データとOHLCデータを表示
                trading_data.display_interpolated_data()
        # 指定した間隔（秒数）待機
        time.sleep(SLEEP_INTERVAL)

if __name__ == '__main__':
    main()


### 価格 ###

,close
2025-01-01 22:40:19.038391,302.9
2025-01-01 22:40:20.416159,302.9
2025-01-01 22:40:21.827265,302.9
2025-01-01 22:40:23.279844,302.9
2025-01-01 22:40:24.651273,302.9
2025-01-01 22:40:26.090133,302.9
2025-01-01 22:40:27.472202,302.9
2025-01-01 22:40:28.870922,302.9
2025-01-01 22:40:30.243092,302.9
2025-01-01 22:40:31.660837,302.9


### エクイティ ###

,buy_and_hold_equity,trading_equity,cash,stock_value,quantity
2025-01-01 22:40:19.038391,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:20.416159,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:21.827265,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:23.279844,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:24.651273,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:26.090133,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:27.472202,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:28.870922,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:30.243092,50000.0,50000.0,50000.0,0.0,0
2025-01-01 22:40:31.660837,50000.0,50000.0,50000.0,0.0,0


### シグナル ###

,buy_signals,sell_signals,hedge_buy_signals,hedge_sell_signals,special_buy_signals,special_sell_signals,sell_exit_signals,buy_exit_signals,emergency_buy_exit_signals,emergency_sell_exit_signals,hedge_buy_exit_signals,hedge_sell_exit_signals,special_buy_exit_signals,special_sell_exit_signals
2025-01-01 22:40:19.038391,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:20.416159,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:21.827265,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:23.279844,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:24.651273,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:26.090133,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:27.472202,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:28.870922,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:30.243092,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0
2025-01-01 22:40:31.660837,0,NaN,0,0,0,0,0,NaN,0,0,0,0,0,0


buy_entry_price
None

sell_entry_price
None

original_entry_price
0.0

special_entry_price
0.0

signal_position_prev2
None

signal_position_prev
None

signal_position
None

signal_position2_prev2
None

signal_position2_prev
None

signal_position2
None



2025-01-01 22:40:45,979 - INFO - 取得した価格: 302.9
2025-01-01 22:40:46,290 - INFO - 取得した価格: 302.9
2025-01-01 22:40:46,603 - INFO - 取得した価格: 302.9
